For this assignment, we had to create an interactive dashboard using the 'buildings' data set. The dashboard required to have 2 visualizations, one a grid map that showed mean square footage with respect to agency and county and another a histogram linked to this grid that showed total square footage acquired that year. 

In [189]:
import pandas as pd
import bqplot
import bqplot.pyplot 
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt

In [190]:
mydata = pd.read_csv("https://uiuc-ischool-dataviz.github.io/is445_AOUAOG_fall2021/week02/data/building_inventory.csv")

In [191]:
mydata= mydata.dropna()#using dropna() function to drop na values and replacing na values in the pivot table by 0
mydata

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975,1975,144,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8777,Southern Illinois University,Southern Illinois University - Edwardsville,3690 Stadium Drive,Edwardsville,62025,Madison,13,Rodney L. Davis,112,Stuart Katie,...,In Use,2013,2013,1746,1,0,0,Education,Utiility & Miscellan,Not provided
8778,Southern Illinois University,Southern Illinois University - Edwardsville,3690 Stadium Drive,Edwardsville,62025,Madison,13,Rodney L. Davis,112,Stuart Katie,...,In Use,2012,2012,2715,1,0,0,Storage,Utiility & Miscellan,Not provided
8779,Southern Illinois University,Southern Illinois University - Edwardsville,65 Circle Drive,Edwardsville,62026,Madison,13,Rodney L. Davis,112,Stuart Katie,...,In Use,2014,2014,31455,4,0,0,Education,Assembly,Not provided
8780,Department of Veterans' Affairs,Chicago Veterans' Home - Cook County,4250 North Oak Park Avenue,Chicago,60634,Cook,5,Mike Quigley,19,Martwick Robert,...,In Progress,0,2015,185525,5,0,0,Health Care,Not provided,Not provided


As suggested by professor Jill Naiman, to implement the heatmap I took help of the pivot table to take the required columns and value. I refered to the post on the slack channel and saw the code from: https://datagy.io/python-pivot-tables/ 

This made my work easier. Specially the process of implementing a heat map. I could extract the fileds I needed as well as carry out the required computation just by executing one line of code  

In [192]:
mypivot = pd.pivot_table(mydata, index = 'Agency Name', columns='County',values = 'Square Footage', aggfunc = 'mean',fill_value=0)

As seen from the above line of code, I took the column as 'County' as required and kept 'Agency Names' as index (Which makes each agency a record). This pair will represent the mean of 'square footage' when a cell is selected on the grid map. As seen previously, I used 'dropna()' to drop na values and in the above pivot table code I have replaced these values with the number 0, so the grid heat map looks asthetically readable and is easier for the user to comprehend and look at. 

First, I wrote the code for implementing the bar plot. The code is relativley simple, with first defining the scale, where the x axis scale is 'ordinal' scale comprised of different years while y-axis represents the total sqaure footage. The x[] & y[] values are actually passed from the user when the user selects the cell on the grid heat map for which the parameters discussed are then represented on the histogram.

In [193]:
# Scale for bar plot
x_scale = bqplot.OrdinalScale()
y_scale = bqplot.LinearScale()

# Axes for bar plot
ax_xb = bqplot.Axis(scale = x_scale, label = 'Year')
ax_yb = bqplot.Axis(scale = y_scale, label = 'Square Footage', orientation = 'vertical')

# Mark
bp = bqplot.pyplot.bar(x = [], y = [], scales={'x': x_scale, 'y': y_scale})

figurebar = bqplot.Figure(marks = [bp], axes=[ax_xb, ax_yb])


Below is the code for implementing the grid heat map and also creating the dashboard with interactivity between the two visualizations. I refered the code from Professor Jill Naiman's Week 7 in class workbook as well as from week 07 prep notebook part-1.
Again here the code is divided into parts where I define the scales first. I have used ordinal scales because we were required to represent categorical variables such as agency name and county and also used the 'RedPu' color scale which I thought looked good and is readable to the user. I defined 3 axes. one is the color axes defiend to the right and the others is the x and y axes. Next, I defined the mark (grid heat map) and made it interactive and defined an anchor color(blue) as well. Next, for the purpose of linkage between the two plots I defined a function for the bar plot that takes in values from user selection to represent the related data on the bar plot. I followed this by defining the action to be taken when a uer selects one of the cells. After selection a label will show the name of county, agency and the mean square footage for the same while the bar plot will represent the years and the total sqaure footage acquired in those years for the user selection. And finally, the dashboard is defined and represented. I tried first using the VBox followed by HBox for the grid heat map and the bar plot but later I settled for a couple of VBoxes because in hindsight it was a better at representing the plots as a whole. 

In [195]:
#Now that I have the required data using pivot, I made the heat map

# 1. data (mypivot)

# 2. scales
col_sc = bqplot.ColorScale(scheme='RedPu')
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale() 

# 3. axis
col_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
x_ax = bqplot.Axis(scale=x_sc, label='County')
y_ax = bqplot.Axis(scale=y_sc, orientation='vertical', label='Agency Name')

# 4. Marks
heat_map = bqplot.GridHeatMap(color=mypivot.values, 
                             row = mypivot.index, column = mypivot.columns, 
                             scales={'color':col_sc, 'row':y_sc, 'column':x_sc}, 
                             interactions={'click':'select'}, 
                             anchor_style={'fill':'blue'})
def barplot_fig(x, y):
    df2 = mydata[(mydata['County'] == y) & (mydata['Agency Name'] == x)]
    df2 = df2[df2['Year Acquired'] != 0]
    df2 = df2.groupby('Year Acquired')['Square Footage'].sum()
    return df2

    
# 5. linking 
mySelectedLabel = ipywidgets.Label()
def on_selected(change):
    if len(change['owner'].selected) == 1: # only allow user to select 1 bin
        i,j = change['owner'].selected[0]
        v = mypivot.values[i,j] # data value in that bin
        mySelectedLabel.value = 'Mean Square Footage for Agency: ' + mypivot.index[i] + ' and County: ' + mypivot.columns[j] + ' = ' + str(v)
        buildings_final = barplot_fig(mypivot.index[i], mypivot.columns[j])
     
        bp.x = buildings_final.index
        bp.y = buildings_final.values
heat_map.observe(on_selected, 'selected')


# 6. figure!
fig = bqplot.Figure(marks=[heat_map], axes=[col_ax, x_ax, y_ax])
fig.layout.min_width='500px'
figurebar.layout.min_width = '500px'
myDashboard = ipywidgets.VBox([mySelectedLabel, fig])
myDashboard_final = ipywidgets.VBox([myDashboard, figurebar])
myDashboard_final